<a href="https://colab.research.google.com/github/Gabikero/C-digo_Maquina/blob/main/Sistema_de_Suegerencia_II.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<center><h1>Sistema de Sugerencias<center>

Filtrado Colaborativo

In [2]:
!wget -O moviedataset.zip https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-ML0101EN-SkillsNetwork/labs/Module%205/data/moviedataset.zip
print('unziping ...')
!unzip -o -j moviedataset.zip 

--2023-03-15 13:08:06--  https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-ML0101EN-SkillsNetwork/labs/Module%205/data/moviedataset.zip
Resolving cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)... 169.45.118.108
Connecting to cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)|169.45.118.108|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 160301210 (153M) [application/zip]
Saving to: ‘moviedataset.zip’

moviedataset.zip    100%[===================>] 152.88M  31.7MB/s    in 5.0s    

2023-03-15 13:08:11 (30.8 MB/s) - ‘moviedataset.zip’ saved [160301210/160301210]

unziping ...
Archive:  moviedataset.zip
  inflating: links.csv               
  inflating: movies.csv              
  inflating: ratings.csv             
  inflating: README.txt              
  inflating: tags.csv        

In [3]:
#Importe de librerias
import pandas as pd
from math import sqrt
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [4]:
#Información de géneros de movies
movies_df = pd.read_csv('/content/movies.csv')
#Historia de usuarios
ratings_df = pd.read_csv('/content/ratings.csv')

movies_df.head()



,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


**Acondicionamos Géneros de Movie**

In [6]:
#Indexar una columna, la llamaremos year, extraer la fecha de la columna title
movies_df['year'] = movies_df.title.str.extract('(\(\d\d\d\d\))',expand=False)
movies_df.head()


,movieId,title,genres,year
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,(1995)
1,2,Jumanji (1995),Adventure|Children|Fantasy,(1995)
2,3,Grumpier Old Men (1995),Comedy|Romance,(1995)
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance,(1995)
4,5,Father of the Bride Part II (1995),Comedy,(1995)


In [7]:
#Removemos el parentesis del año
movies_df['year'] = movies_df.year.str.extract('(\d\d\d\d)',expand=False)
movies_df.head()


,movieId,title,genres,year
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,1995
1,2,Jumanji (1995),Adventure|Children|Fantasy,1995
2,3,Grumpier Old Men (1995),Comedy|Romance,1995
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance,1995
4,5,Father of the Bride Part II (1995),Comedy,1995


In [8]:
#Remover del la columna title el año con la función replace un espacio en blanco en el lugar del año
movies_df['title'] = movies_df.title.str.replace('(\(\d\d\d\d\))', '')
movies_df.head()


<ipython-input-8-8ab5817656e9>:2: FutureWarning: The default value of regex will change from True to False in a future version.
  movies_df['title'] = movies_df.title.str.replace('(\(\d\d\d\d\))', '')


,movieId,title,genres,year
0,1,Toy Story,Adventure|Animation|Children|Comedy|Fantasy,1995
1,2,Jumanji,Adventure|Children|Fantasy,1995
2,3,Grumpier Old Men,Comedy|Romance,1995
3,4,Waiting to Exhale,Comedy|Drama|Romance,1995
4,5,Father of the Bride Part II,Comedy,1995


In [9]:
#Aplicar la función strip para eliminar el espacio en blanco que aparezca al final del registro de title
movies_df['title'] = movies_df['title'].apply(lambda x: x.strip())
movies_df.head()

,movieId,title,genres,year
0,1,Toy Story,Adventure|Animation|Children|Comedy|Fantasy,1995
1,2,Jumanji,Adventure|Children|Fantasy,1995
2,3,Grumpier Old Men,Comedy|Romance,1995
3,4,Waiting to Exhale,Comedy|Drama|Romance,1995
4,5,Father of the Bride Part II,Comedy,1995


In [10]:
#Modificar los registros genres que tenga separados por |, en una lista
movies_df['genres'] = movies_df.genres.str.split('|')
movies_df.head()

,movieId,title,genres,year
0,1,Toy Story,"[Adventure, Animation, Children, Comedy, Fantasy]",1995
1,2,Jumanji,"[Adventure, Children, Fantasy]",1995
2,3,Grumpier Old Men,"[Comedy, Romance]",1995
3,4,Waiting to Exhale,"[Comedy, Drama, Romance]",1995
4,5,Father of the Bride Part II,[Comedy],1995


In [11]:
ratings_df.head()

,userId,movieId,rating,timestamp
0,1,169,2.5,1204927694
1,1,2471,3.0,1204927438
2,1,48516,5.0,1204927435
3,2,2571,3.5,1436165433
4,2,109487,4.0,1436165496


**Matriz Movie**

In [12]:
#Borramos la columna timestamp
ratings_df = ratings_df.drop('timestamp', 1)
ratings_df.head()

<ipython-input-12-235753475d73>:2: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  ratings_df = ratings_df.drop('timestamp', 1)


,userId,movieId,rating
0,1,169,2.5
1,1,2471,3.0
2,1,48516,5.0
3,2,2571,3.5
4,2,109487,4.0


**<h2>Filtrado Colaborativo**

**Calificación de Usuario**

In [13]:
userInput = [
            {'title':'Breakfast Club, The', 'rating':5},
            {'title':'Toy Story', 'rating':3.5},
            {'title':'Jumanji', 'rating':2},
            {'title':"Pulp Fiction", 'rating':5},
            {'title':'Akira', 'rating':4.5},
            {'title': 'Avatar', 'rating': 5}
         ] 
#Generamos un DataFrames a partir de la lista
inputMovies = pd.DataFrame(userInput)
inputMovies

,title,rating
0,"Breakfast Club, The",5.0
1,Toy Story,3.5
2,Jumanji,2.0
3,Pulp Fiction,5.0
4,Akira,4.5
5,Avatar,5.0


In [14]:
#Filtramos las salidas de peliculas en la columna title
#isin comprueba los valores de entrada
#tolist convierte en lista
inputId = movies_df[movies_df['title'].isin(inputMovies['title'].tolist())]

#Fusionamos inpuId-inputMovies
inputMovies = pd.merge(inputId, inputMovies)

#Descartar información que no usaremos del marco de datos de entrada
inputMovies = inputMovies.drop('genres', 1).drop('year', 1)

inputMovies

<ipython-input-14-f859429ad23a>:10: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  inputMovies = inputMovies.drop('genres', 1).drop('year', 1)
<ipython-input-14-f859429ad23a>:10: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  inputMovies = inputMovies.drop('genres', 1).drop('year', 1)


,movieId,title,rating
0,1,Toy Story,3.5
1,2,Jumanji,2.0
2,296,Pulp Fiction,5.0
3,1274,Akira,4.5
4,1968,"Breakfast Club, The",5.0
5,72998,Avatar,5.0


**Usuarios que vieron la peliculas de la matriz de entrada**

Con los ID de películas en nuestra entrada, ahora podemos obtener el subconjunto de usuarios que vieron y revisaron las películas en nuestra entrada.

In [18]:
#Filtrado de usuarios que vieron las películas
userSubset = ratings_df[ratings_df['movieId'].isin(inputMovies['movieId'].tolist())]
userSubset.head()

,userId,movieId,rating
19,4,296,4.0
441,12,1968,3.0
479,13,2,2.0
531,13,1274,5.0
681,14,296,2.0


In [19]:
#Agrupamos las filas por usuario
userSubsetGroup = userSubset.groupby(['userId'])

**Monitoriamos un usuario con Id: 1200**

In [22]:
userSubsetGroup.get_group(1200)

,userId,movieId,rating
110608,1200,1,2.0
110609,1200,2,1.0
110621,1200,296,4.0
110717,1200,1968,3.0


Ordenar los grupos para que los usuarios que comparten la mayoría de las películas en común con la entrada tengan mayor prioridad.

In [23]:
#Ordenarlo para que los usuarios con la película más en común con la entrada tengan prioridad
userSubsetGroup = sorted(userSubsetGroup,  key=lambda x: len(x[1]), reverse=True)

In [25]:
#Vemos a la primera usuaria
userSubsetGroup[0:3]

[(106,        userId  movieId  rating
  9083      106        1     2.5
  9084      106        2     3.0
  9115      106      296     3.5
  9198      106     1274     3.0
  9238      106     1968     3.5
  10041     106    72998     4.0), (815,        userId  movieId  rating
  73747     815        1     4.5
  73748     815        2     3.0
  73922     815      296     5.0
  74362     815     1274     3.0
  74678     815     1968     4.5
  77813     815    72998     4.0), (1040,        userId  movieId  rating
  96689    1040        1     3.0
  96690    1040        2     1.5
  96733    1040      296     3.5
  96859    1040     1274     3.0
  96922    1040     1968     4.0
  97457    1040    72998     4.0)]

In [27]:
#Seleccionaremos un subconjunto de usuarios para iterar.
# Este límite se impone porque no queremos perder demasiado tiempo 
#revisando a cada usuario.

In [26]:
userSubsetGroup = userSubsetGroup[0:100]

**Correlación de Pearson**

Calculamos la correlación de Pearson entre el usuario de entrada y el grupo de subconjuntos, y la almacenamos en un diccionario, donde la clave es la identificación del usuario y el valor es el coeficiente.

In [31]:
#Almacene la Correlación de Pearson en un diccionario, donde la clave es el IDusuario y el valor es el coeficiente
dict_pearson = {}

#Aplicamos un for para cada grupo de usuario
for name, group in userSubsetGroup:
    #Comencemos por ordenar la entrada y el grupo de usuarios actual
    group = group.sort_values(by='movieId') 
    inputMovies = inputMovies.sort_values(by='movieId')
    #Obtener N, es decir longitud para la formula
    nRatings = len(group)
    #Se buscan los puntajes de las peliculas en comun
    temp_df = inputMovies[inputMovies['movieId'].isin(group['movieId'].tolist())]
    #Guardar en una variable de búfer temporal en formato de lista para facilitar los cálculos
    tempRatingList = temp_df['rating'].tolist()
    #Registrar las reseñas de los grupos de usuarios actuales en un formato de lista
    tempGroupList = group['rating'].tolist()
    #Ahora calculemos la correlación de Pearson entre dos usuarios, llamados x e y.
    Sxx = sum([i**2 for i in tempRatingList]) - pow(sum(tempRatingList),2)/float(nRatings)
    Syy = sum([i**2 for i in tempGroupList]) - pow(sum(tempGroupList),2)/float(nRatings)
    Sxy = sum( i*j for i, j in zip(tempRatingList, tempGroupList)) - sum(tempRatingList)*sum(tempGroupList)/float(nRatings)
    
    #Si es denominador es igual a cero se divide, sino 0 la correlación
    if Sxx != 0 and Syy != 0:
        dict_pearson[name] = Sxy/sqrt(Sxx*Syy)
    else:
        dict_pearson[name] = 0


In [32]:
dict_pearson.items()

dict_items([(106, 0.6298366572977737), (815, 0.5428101483418095), (1040, 0.9460998335825315), (1130, 0.0), (1502, 0.7416198487095665), (1625, 0.7416198487095665), (1950, 0.3015113445777637), (2791, 0.6005286407578742), (3186, 0.5738898237297515), (3734, -0.10562681853292959), (4099, 0.1088931012960942), (4208, 0.39192475766909723), (4292, 0.6929023281557342), (4415, 0.0), (4586, -0.7565779193127061), (4818, 0.4189652907981962), (5104, 0.7043883360626504), (5547, 0.07872958216222171), (6082, 0.06998656386948264), (6530, -0.3656362120635646), (7235, 0.6937141194961621), (7403, 0.07312724241271541), (8008, -0.29545454545454464), (8086, 0.6527912098338671), (9101, 0.17516226243634272), (10863, 0.3296171265436097), (11399, 0.31788197660287687), (12069, 0.2277867258047116), (12921, 0.5237849266164991), (13142, 0.299521148936577), (13260, 0.7727272727272727), (14588, -0.02799462554779119), (14984, 0.5330017908890288), (15466, 0.6784005252999683), (15834, 0.32639560491693353), (16456, 0.533001

In [47]:
#Indexamos similarityIndex en el DATAFRANES Pearson
pearsonDF = pd.DataFrame.from_dict(dict_pearson, orient='index')
pearsonDF.columns= ['similarityIndex']

#Indexamos la columna userId
pearsonDF['userId']= pearsonDF.index
pearsonDF.index= range(len(pearsonDF))
pearsonDF.head()

,similarityIndex,userId
0,0.629837,106
1,0.542810,815
2,0.946100,1040
3,0.000000,1130
4,0.741620,1502


**Top de usuarios similares al usauario de entrada**

In [49]:
#Lo hacemos para los primeros 50 usuarios de manera descendente
topUsers=pearsonDF.sort_values(by='similarityIndex', ascending=False)[0:50]
topUsers.head()

,similarityIndex,userId
45,0.986928,20650
2,0.946100,1040
59,0.926872,25799
79,0.867328,35964
98,0.866025,44758


**Valoración de usuarios seleccionados a todas las películas.**

In [50]:
topUsersRating=topUsers.merge(ratings_df, left_on='userId', right_on='userId', how='inner')
topUsersRating.head()

,similarityIndex,userId,movieId,rating
0,0.986928,20650,1,3.0
1,0.986928,20650,2,2.5
2,0.986928,20650,29,4.0
3,0.986928,20650,32,4.0
4,0.986928,20650,39,1.5


** Usuarios similares a las películas candidatas para el usuario de entrada**

Se basa en multiplicar dos columnas, luego agrupando el marco de datos por movieId y luego dividiendo dos columnas.

In [52]:
#Se multiplica similarity y ratings de los usuarios
topUsersRating['weightedRating'] = topUsersRating['similarityIndex']*topUsersRating['rating']
topUsersRating.head()

,similarityIndex,userId,movieId,rating,weightedRating
0,0.986928,20650,1,3.0,2.960783
1,0.986928,20650,2,2.5,2.467319
2,0.986928,20650,29,4.0,3.947710
3,0.986928,20650,32,4.0,3.947710
4,0.986928,20650,39,1.5,1.480391


In [53]:
#Se agrupan los usuario por ID y luego se hace la sumatoria de los rating
tempTopUsersRating = topUsersRating.groupby('movieId').sum()[['similarityIndex','weightedRating']]
tempTopUsersRating.columns = ['sum_similarityIndex','sum_weightedRating']
tempTopUsersRating.head()

,sum_similarityIndex,sum_weightedRating
movieId,,
1,33.572077,116.182475
2,33.572077,83.679914
3,9.044298,21.570425
4,1.473268,1.602659
5,9.764355,22.985765


In [55]:
#Crea un DATAFRAMES vacío que se usará para la matriz recomendación
recommendation_df = pd.DataFrame()
#Calculamos el promedio ponderado en una columna que llamaremos 'weighted average recommendation score'
recommendation_df['weighted average recommendation score'] = tempTopUsersRating['sum_weightedRating']/tempTopUsersRating['sum_similarityIndex']
recommendation_df['movieId'] = tempTopUsersRating.index
recommendation_df.head()

,weighted average recommendation score,movieId
movieId,,
1,3.460688,1
2,2.492545,2
3,2.384975,3
4,1.087826,4
5,2.354048,5


**Ordenamos las primeras 20 peliculas recomendadas**

In [56]:
recommendation_df = recommendation_df.sort_values(by='weighted average recommendation score', ascending=False)
recommendation_df.head(20)

,weighted average recommendation score,movieId
movieId,,
118702,5.0,118702
2585,5.0,2585
72899,5.0,72899
26801,5.0,26801
73501,5.0,73501
41627,5.0,41627
2920,5.0,2920
7792,5.0,7792
7815,5.0,7815


***Películas recomendadas***

In [58]:
movies_df.loc[movies_df['movieId'].isin(recommendation_df.head(10)['movieId'].tolist())]

,movieId,title,genres,year
2501,2585,"Lovers of the Arctic Circle, The (Los Amantes ...","[Drama, Romance]",1998
2835,2920,Children of Paradise (Les enfants du paradis),"[Drama, Romance]",1945
5669,5767,Teddy Bear (Mis),"[Comedy, Crime]",1981
7485,7792,"Parallax View, The",[Thriller],1974
7494,7815,True Stories,"[Comedy, Musical]",1986
9064,26801,Dragon Inn (Sun lung moon hak chan),[Action],1992
10662,41627,Samurai Rebellion (Jôi-uchi: Hairyô tsuma shim...,"[Action, Drama]",1967
14592,72899,Blissfully Yours (Sud sanaeha),"[Drama, Romance]",2002
14737,73501,Pekka ja Pätkä Suezilla,[Comedy],1958
25460,118702,Unbroken,"[Drama, War]",2014


**Conclusión:**

Tenemos el mismo conjunto de entrada y si aplicamos el método de filtrado colaborativo observamos que hay una diferencias en las peliculas recomendadas, con respecto a el método basado en contenido. 

Esto se debe a que el enfoque colaborativo, usa como enfasis las similitudes entre usuarios vecinos o comúnes al usuario en cuentión. Mientras que el filtrado en base a contenidos, depende del perfil del usuario, sus intereses y como estos se relacionan en categorias con otras películas en un grado de ponderación que le asigna jerarquia de relevancia.

